<a href="https://colab.research.google.com/github/moter1026/review_Kinopoisk_lab5/blob/main/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортирую необходимые библиотеки и получаю все отзывы из заранее созданного csv файла

In [4]:
import torch
from torch.utils.data import Dataset
from typing import Tuple, Any
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import math
import random

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from google.colab import drive
import sys
# import copy_dataset_random
# from lab4 import make_dataFrame

drive.mount('/content/drive')
sys.path.insert(0, "/content/drive/MyDrive/lab5_drive")
%cd /content/drive/MyDrive/lab5_drive/

allTextsDf = pd.read_csv("/content/drive/MyDrive/lab5_drive/big_description.csv")
print(allTextsDf)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/lab5_drive
                                           text of file class
0     Форрест Гамп\n\n\n\nФоррест Гамп – не совсем о...  good
1     Форрест Гамп\n\n\n\nСразу хочу сказать, что по...  good
2     Форрест Гамп\n\n\n\nКогда я сажусь смотреть ка...  good
3     Форрест Гамп\n\n\n\nВпервые посмотрев этот фил...  good
4     Форрест Гамп\n\n\n\nНе поспоришь - Том Хэнкс г...  good
...                                                 ...   ...
2123  Зеленая миля\n\n\n\nФильм не оправдал ожиданий...   bad
2124  Зеленая миля\n\n\n\nФактически фильм начинаетс...   bad
2125  Зеленая миля\n\n\n\n'Уж сколько раз твердили м...   bad
2126  Зеленая миля\n\n\n\nНе буду повторяться за пре...   bad
2127  Зеленая миля\n\n\n\nЧего ожидаешь, когда садиш...   bad

[2128 rows x 2 columns]


Считаю количество хороших и плохих комментариев

In [5]:
value_counts = allTextsDf['class'].value_counts()

countGoodTexts = value_counts.get("good", 0)
countBadTexts = value_counts.get("bad", 0)

разделяю списки со всеми хорошими и плохими элементами в пропорциях 80:10:10,
где 80% - это для обучения, 10% - для тестов, 10% - для валинации. Сразу перемешиваю все элементы в DataFram-ах

In [9]:
badTexts = allTextsDf[allTextsDf["class"] != "good"]
goodTexts = allTextsDf[allTextsDf["class"] != "bad"]

AllTextsForLearning = 0
AllTextsForTest = 0
AllTextsForValid = 0

goodTextsForLearning = goodTexts[:math.floor(countGoodTexts*0.8)]
badTextsForLearning = badTexts[:math.floor(countBadTexts*0.8)]
AllTextsForLearning = pd.concat([goodTextsForLearning, badTextsForLearning], ignore_index = True).sample(frac=1, random_state = random.randrange(0, 10000))
print(len(AllTextsForLearning))


goodTextsForTest = goodTexts[math.floor(countGoodTexts*0.8) : math.floor(countGoodTexts*0.8) + math.floor(countGoodTexts*0.1)]
badTextsForTest = badTexts[math.floor(countBadTexts*0.8) : math.floor(countBadTexts*0.8) + math.floor(countBadTexts*0.1)]
AllTextsForTest = pd.concat([goodTextsForTest, badTextsForTest], ignore_index = True).sample(frac=1, random_state = random.randrange(0, 10000))

goodTextsForValid = goodTexts[math.floor(countGoodTexts*0.8) + math.floor(countGoodTexts*0.1) :]
badTextsForValid = badTexts[math.floor(countBadTexts*0.8) + math.floor(countBadTexts*0.1) :]
AllTextsForTest = pd.concat([goodTextsForValid, badTextsForValid], ignore_index = True).sample(frac=1, random_state = random.randrange(0, 10000))


1701


Создаю класс для собственного датасета

In [10]:
class TextDataset(Dataset):
  def __init__(self, data: pd.DataFrame, transform: Any=None) -> None:
    self.data = data
    self.transform = transform

  def __len__(self) -> int:
    return len(self.data)

  def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
    text = self.data.iloc[index, 0]
    type_class = self.data.iloc[index, 1]
    if self.transform:
      text = self.transform(self.data.iloc[index])

    return text, type_class


('Игра престолов\n\n\n\nКроме дерьма и половых органов этот сериал, возможно, привлекает своим нестандартным сюжетом, который отсекает всякую надежду и веру человека в лучшее.\n\n\nЧтобы оправдать свою нравственную деградацию, нужно показать, что раньше всё было гораздо жестче и развратнее, а мораль никогда не приветствовалась. Современное общество уже скатилось к этому самому Средневековью, именно скатилось, потому что нравственность и мораль не придумала кучка дегенератов в 21 веке. Почему? Да, потому что люди потеряли смысл своего существования, наплевали на устои и традиции по которым веками жили их предки. И их образ жизни действительно стал подобным героям этого сериала, у кого то в большей степени, у кого то в меньшей. Наверно смысл этого сериала сроден со смыслом фильма Феррери, но в отличие от фильма языческий Рим не раскололся в своём ликующем веселье, он продолжает существовать и жить в сердцах современных людей., а зло остаётся безнаказанным. Могу сказать одно, теряя веру в